In [1]:
import vk_api, json
from vk_api.longpoll import VkLongPoll, VkEventType
from vk_api.keyboard import VkKeyboard, VkKeyboardColor
from vk_api.bot_longpoll import VkBotLongPoll, VkBotEventType
import requests
import sqlite3
from collections import defaultdict
from vk_api.utils import get_random_id

In [2]:
token = '9d662b711c8bb91e64dbafc5cb5088218eab3b578758d4a2c878b4e2ce11d8a6e3bc869d65ab99708e1ef'
group_id = '201615991'
version = '5.103'
group_name_short = 'bec204'

In [3]:
session = vk_api.VkApi(token=token)
longpoll = VkBotLongPoll(session, group_id)

In [4]:
def send_message_to_user (user_id, peer_id, message, keyboard=None): #сделать проще!
    post = {
        "user_id": user_id,
        "random_id": get_random_id(),
        "peer_id": peer_id,
        "message": message,
        "random_id": 0
    }
    if keyboard != None:
        post = {
        "user_id": user_id,
        "random_id": get_random_id(),
        "peer_id": peer_id,
        "message": message,
        "random_id": 0,
        "keyboard" : keyboard.get_keyboard()
        }
    session.method('messages.send', post)

In [5]:
def send_message_to_chat (chat_id, peer_id, message, keyboard=None): #сделать проще!
    post = {
        "chat_id": chat_id,
        "random_id": get_random_id(),
        "peer_id": peer_id,
        "message": message,
        "random_id": 0
    }
    if keyboard != None:
        post = {
        "chat": chat_id,
        "random_id": get_random_id(),
        "peer_id": peer_id,
        "message": message,
        "random_id": 0,
        "keyboard" : keyboard.get_keyboard()
        }
    session.method('messages.send', post)

In [6]:
def get_conversations(offset, count, filt):
    post = {
        "offset": offset,
        "count": count,
        "filter": filt
    }
    session.method("messages.getConversations", post)

In [7]:
def get_Conversations (peer_ids): 
    post = {
        "peer_ids": peer_ids
    }
    session.method("messages.getConversations", post)

In [8]:
def vk_download(method, parameters, fields=''):
    url = 'https://api.vk.com/method/' + method + '?' + parameters + '&fields=' + fields + '&access_token=' + token + '&v=' + version
    response = requests.get(url)
    infa = response.json()
    return infa

In [9]:
# подключаем базу данных
conn = sqlite3.connect('chats.db')

# курсор для работы с таблицами
cursor = conn.cursor()

try:
    # sql запрос для создания таблицы
    query = "CREATE TABLE \"chats\" (\"ID\" INTEGER UNIQUE, \"group\" TEXT, PRIMARY KEY (\"ID\"))"
    # исполняем его –> ура, теперь у нас есть таблица, куда будем все сохранять!
    cursor.execute(query)
except:
    pass

In [10]:
#добавить группу в БД
def add_group(chat_id, group):
    with sqlite3.connect('chats.db') as con:
        cursor = con.cursor()
        cursor.execute('INSERT INTO chats (ID, group) VALUES (?, ?)',
                       (chat_id, group))
        con.commit()

In [11]:
def normal(st_group):
    groups = defaultdict()
    for elem in st_group:
        chat_id = elem[1]
        group = elem[2]
        groups[group] = chat_id
    return groups
    
def choose_group():
    with sqlite3.connect('chats') as con:
        cursor = con.cursor()
        cursor.execute('SELECT * FROM chats')
        groups = normal(cursor.fetchall())
    return groups

In [ ]:
while True:
    
    try:
        for event in longpoll.listen():
            if event.type == VkBotEventType.MESSAGE_NEW:
                text = event.object['text'].lower()
                peer_id = event.obj.peer_id
        
                if event.from_user:#Если написали в ЛС
                    user_id = event.object['from_id']
                    
                    if text == 'начать':
                        keyboard_start = VkKeyboard(one_time = False, inline = True)
                        keyboard_start.add_button('Добавить беседу', VkKeyboardColor.POSITIVE)
                        keyboard_start.add_line()
                        keyboard_start.add_button('Отправить сообщение', VkKeyboardColor.SECONDARY)

                        send_message_to_user(user_id, peer_id, 'Что сделать?', keyboard_start)

                    elif text == 'отправить сообщение':
                        chosen_groups = []
                        keyboard_choose_group = VkKeyboard(one_time = False)
                        groups = choose_group()
                        for elem in groups:
                            keyboard_choose_group.add_button(elem[1])
                        keyboard_choose_group.add_button('Всё выбрано')
                        send_message_to_user(user_id, peer_id, 'Выберите группу', keyboard_choose_group)

                    elif text in list(groups.keys()):
                        chosen_groups.append(groups[text])
                        send_message(user_id, peer_id, 'Ещё одну?', keyboard_choose_group)
                    elif text == 'Всё выбрано':
                        send_message_to_user(user_id, peer_id, 'Введите сообщение')
                        for event in VkBotLongPoll(session).listen():
                            if event.type == VkBotEventType.MESSAGE_NEW:
                                text = event.object['text'].lower()
                                user_id = event.object['from_id']
                        for elem in chosen_groups:
                            send_message_to_user(elem, peer_id, text)

                     


                elif event.from_chat:
                    chat_id = event.chat_id
                    if text == 'регистрация':
                        keyboard_reg = VkKeyboard(one_time = False, inline = True)
                        keyboard_reg.add_button('БЭК204', VkKeyboardColor.POSITIVE)
                        keyboard_reg.add_line()
                        keyboard_reg.add_button('БЭК201', VkKeyboardColor.SECONDARY)

                        send_message_to_chat(chat_id, peer_id, 'Выберите свою группу', keyboard_reg)
            
#             if event.type == VkBotEventType.MESSAGE_EVENT: #callback-кнопки   
#                 print(event)
                    
                    
                    
                    elif text == f'@{group_name_short}'+ ' бэк204':
                        
                        group = 'БЭК204'
                        add_group(chat_id, group)
                        send_message_to_chat(chat_id, peer_id, 'Принято')

                    elif text == 'бэк201':
                        group = 'БЭК201'
                        add_group(chat_id, group)
                        send_message_to_chat(chat_id, peer_id, 'Принято')

                        print(chats)

                
    except requests.exceptions.ReadTimeout as timeout:
        continue          
                
#         elif event.from_chat: #Если написали в Беседе
#             chat_id = event.chat_id
#             if text = 'Зарегистрировать беседу': #ветка с регистрацией в БД
                

In [ ]:
# if event_chat.type

In [ ]:
event.type == VkBotEventType.MESSAGE_EVENT